In [1]:
from clustering_script import *
from graph_generating_script import *
from choosing_best_num_algorithms import *

In [3]:
from tqdm import tqdm 

class BestNumExperiment():
    def __init__(self, graph_params:pd.DataFrame):
        self.graph_params = graph_params
        self.graph_posdfs = {}

    def calculate_posdfs(self, from_file=None):
        if from_file==None:
            for _, row in tqdm(self.graph_params.iterrows(), total=self.graph_params.shape[0]):
                print(row['graph_id'])
                (G, true_labels) = generate_G_randomized(int(row['size']), int(row['no_comms']), row['inside_prob'], row['outside_prob'])
                self.graph_posdfs[row['graph_id']] = {}
                assor = nx.numeric_assortativity_coefficient(G, "community")
                self.graph_posdfs[row['graph_id']]['assortativity'] = assor
                layout_names = ['kamada_kawai', 'spring', 'davidson_harel', 'drl', 'fruchterman_reingold', 'graphopt', 'lgl','mds']
                for layout_name in layout_names:
                    posdf = posdf_from_layout(G, layout_name)
                    
                    # Store the posdf DataFrame in the nested dictionary
                    self.graph_posdfs[row['graph_id']][layout_name] = posdf
        else:
            self.graph_posdfs
    

    def make_experiment(self, best_num_algo_name):
        #iterating through all the graphs
        results = pd.DataFrame(columns=['graph_id', 'assortativity', 'layout_name', 'no_communities', 'calculated_bestnum'])
        for _, row in tqdm(self.graph_params.iterrows(), total=self.graph_params.shape[0]):
        # for _, row in self.graph_params.iterrows():
            # (G, true_labels) = generate_G_randomized(int(row['size']), int(row['no_comms']), row['inside_prob'], row['outside_prob'])
            # assor = nx.numeric_assortativity_coefficient(G, "community")
            layout_names = ['kamada_kawai', 'spring', 'davidson_harel', 'drl', 'fruchterman_reingold', 'graphopt', 'lgl','mds']
            for layout_name in layout_names:
                # posdf = posdf_from_layout(G, layout_name)
                posdf = self.graph_posdfs[row['graph_id']][layout_name]
                if best_num_algo_name == 'gap_statistic':
                    (best_num, _) = gap_statistic_best_num(posdf)
                elif best_num_algo_name == 'elbow_method':
                    (best_num, _) = elbow_method_best_num(posdf)
                elif best_num_algo_name == 'silhouette':
                    (best_num, _) = silhouette_best_num(posdf)
                elif best_num_algo_name == 'calinski_harabasz':
                    (best_num, _) = calinski_harabasz_best_num(posdf)
                elif best_num_algo_name == '50_mix_ch_elbow':
                    best_num = mix_ch_elbow(posdf, .5, .5)
                elif best_num_algo_name == '75_mix_ch_elbow':
                    best_num = mix_ch_elbow(posdf, .75, .25)
                elif best_num_algo_name == 'BIC':
                    (best_num, _) = BIC_best_num(posdf)
                else:
                    raise ValueError('Incorrect algorith name, probably a typo')

                new_row = {'graph_id':[int(row['graph_id'])], 
                           'assortativity': [self.graph_posdfs[row['graph_id']]['assortativity']], 
                           'size' : [int(row['size'])],
                           'layout_name': [layout_name], 
                           'no_communities': [int(row['no_comms'])],
                            'calculated_bestnum':[int(best_num)]}
                # print(new_row)
                results = pd.concat([results, pd.DataFrame(new_row)])
                
        if best_num_algo_name == 'gap_statistic':
            self.gap= results
        elif best_num_algo_name == 'elbow_method':
            self.elbow = results
        elif best_num_algo_name == 'silhouette':
            self.silhouette = results
        elif best_num_algo_name == 'calinski_harabasz':
            self.ch = results
        elif best_num_algo_name == '50_mix_ch_elbow':
            self.mix_ch_elbow_50 = results
        elif best_num_algo_name == '75_mix_ch_elbow':
            self.mix_ch_elbow_75 = results
        elif best_num_algo_name == 'BIC':
            self.bic = results


# Calculated for all nodes

In [4]:
import warnings
warnings.filterwarnings('ignore')

graph_params = pd.read_excel(r'params\50_graph_params.xlsx')

ex1 = BestNumExperiment(graph_params)
ex1.calculate_posdfs()

size='50'
ex1.make_experiment('gap_statistic')
ex1.gap.to_excel(fr'data\{size}_gap_results.xlsx', index=False)
ex1.make_experiment('elbow_method')
ex1.elbow.to_excel(fr'data\{size}_elbow_results.xlsx', index=False)
ex1.make_experiment('silhouette')
ex1.silhouette.to_excel(fr'data\{size}_silhouette_results.xlsx', index=False)
ex1.make_experiment('calinski_harabasz')
ex1.ch.to_excel(fr'data\{size}_ch_results.xlsx', index=False)
ex1.make_experiment('50_mix_ch_elbow')
ex1.mix_ch_elbow_50.to_excel(fr'data\{size}_50_mix_ch_elbow_results.xlsx', index=False)
ex1.make_experiment('75_mix_ch_elbow')
ex1.mix_ch_elbow_75.to_excel(fr'data\{size}_75_mix_ch_elbow_results.xlsx', index=False)


graph_params = pd.read_excel(r'params\80_graph_params.xlsx')

ex1 = BestNumExperiment(graph_params)
ex1.calculate_posdfs()

size='80'
ex1.make_experiment('gap_statistic')
ex1.gap.to_excel(fr'data\{size}_gap_results.xlsx', index=False)
ex1.make_experiment('elbow_method')
ex1.elbow.to_excel(fr'data\{size}_elbow_results.xlsx', index=False)
ex1.make_experiment('silhouette')
ex1.silhouette.to_excel(fr'data\{size}_silhouette_results.xlsx', index=False)
ex1.make_experiment('calinski_harabasz')
ex1.ch.to_excel(fr'data\{size}_ch_results.xlsx', index=False)
ex1.make_experiment('50_mix_ch_elbow')
ex1.mix_ch_elbow_50.to_excel(fr'data\{size}_50_mix_ch_elbow_results.xlsx', index=False)
ex1.make_experiment('75_mix_ch_elbow')
ex1.mix_ch_elbow_75.to_excel(fr'data\{size}_75_mix_ch_elbow_results.xlsx', index=False)

graph_params = pd.read_excel(r'params\100_graph_params.xlsx')

ex1 = BestNumExperiment(graph_params)
ex1.calculate_posdfs()

size='100'
ex1.make_experiment('gap_statistic')
ex1.gap.to_excel(fr'data\{size}_gap_results.xlsx', index=False)
ex1.make_experiment('elbow_method')
ex1.elbow.to_excel(fr'data\{size}_elbow_results.xlsx', index=False)
ex1.make_experiment('silhouette')
ex1.silhouette.to_excel(fr'data\{size}_silhouette_results.xlsx', index=False)
ex1.make_experiment('calinski_harabasz')
ex1.ch.to_excel(fr'data\{size}_ch_results.xlsx', index=False)
ex1.make_experiment('50_mix_ch_elbow')
ex1.mix_ch_elbow_50.to_excel(fr'data\{size}_50_mix_ch_elbow_results.xlsx', index=False)
ex1.make_experiment('75_mix_ch_elbow')
ex1.mix_ch_elbow_75.to_excel(fr'data\{size}_75_mix_ch_elbow_results.xlsx', index=False)


graph_params = pd.read_excel(r'params\150_graph_params.xlsx')

ex1 = BestNumExperiment(graph_params)
ex1.calculate_posdfs()

size='150'
ex1.make_experiment('gap_statistic')
ex1.gap.to_excel(fr'data\{size}_gap_results.xlsx', index=False)
ex1.make_experiment('elbow_method')
ex1.elbow.to_excel(fr'data\{size}_elbow_results.xlsx', index=False)
ex1.make_experiment('silhouette')
ex1.silhouette.to_excel(fr'data\{size}_silhouette_results.xlsx', index=False)
ex1.make_experiment('calinski_harabasz')
ex1.ch.to_excel(fr'data\{size}_ch_results.xlsx', index=False)
ex1.make_experiment('50_mix_ch_elbow')
ex1.mix_ch_elbow_50.to_excel(fr'data\{size}_50_mix_ch_elbow_results.xlsx', index=False)
ex1.make_experiment('75_mix_ch_elbow')
ex1.mix_ch_elbow_75.to_excel(fr'data\{size}_75_mix_ch_elbow_results.xlsx', index=False)

  0%|          | 0/33 [00:00<?, ?it/s]

0.0


  3%|▎         | 1/33 [00:10<05:28, 10.28s/it]

1.0


  6%|▌         | 2/33 [00:17<04:23,  8.50s/it]

...

29.0


 96%|█████████▋| 27/28 [6:10:14<01:23, 83.68s/it]

30.0


100%|██████████| 28/28 [01:56<00:00,  4.16s/it]


<Figure size 800x550 with 0 Axes>

In [6]:
ex1.graph_params.columns

Index(['graph_id', 'size', 'no_comms', 'inside_prob', 'outside_prob'], dtype='object')

# Narrowed down experiment

In [19]:
[5]*6

[5, 5, 5, 5, 5, 5]

In [29]:
import random
graph_params = {
    'inside_prob': [0.5, 0.5, 0.4, 0.4, 0.4, 0.7, 0.7, 0.7, 0.6, 0.3, 0.3]*6,
    'outside_prob': [0.01, 0.02, 0.01, 0.05, 0.07, 0.01, 0.02, 0.05, 0.01, 0.02, 0.05]*6
}
# n- len of insie prob
n= 11
graph_params = pd.DataFrame(graph_params)
no_comms = [random.uniform(2, 7) for _ in range(n)]*6
graph_params['no_comms'] = no_comms
# graph_params['size'] = 300
graph_params['size'] = [100]*n +[200]*n + [300]*n+[400]*n+[500]*n+[600]*n
graph_params['graph_id']= list(range(len(graph_params)))

In [11]:
from tqdm import tqdm 

class NarrowedBestNumExperiment():
    def __init__(self, graph_params:pd.DataFrame):
        self.graph_params = graph_params
        self.graph_posdfs = {}

    def calculate_posdfs_grahopt(self, from_file=None):
        if from_file==None:
            for _, row in tqdm(self.graph_params.iterrows(), total=self.graph_params.shape[0]):
                print(row['graph_id'])
                (G, true_labels) = generate_G_randomized(int(row['size']), int(row['no_comms']), row['inside_prob'], row['outside_prob'])
                self.graph_posdfs[row['graph_id']] = {}
                assor = nx.numeric_assortativity_coefficient(G, "community")
                self.graph_posdfs[row['graph_id']]['assortativity'] = assor
                # layout_names = ['kamada_kawai', 'spring', 'davidson_harel', 'drl', 'fruchterman_reingold', 'graphopt', 'lgl','mds']
                # for layout_name in layout_names:
                posdf = posdf_from_layout(G, 'graphopt')
                    
                # Store the posdf DataFrame in the nested dictionary
                self.graph_posdfs[row['graph_id']]['graphopt'] = posdf
        else:
            self.graph_posdfs
    

    def make_experiment(self, best_num_algo_name):
        #iterating through all the graphs
        results = pd.DataFrame(columns=['graph_id', 'assortativity', 'layout_name', 'no_communities', 'calculated_bestnum'])
        for _, row in tqdm(self.graph_params.iterrows(), total=self.graph_params.shape[0]):
        # for _, row in self.graph_params.iterrows():
            # (G, true_labels) = generate_G_randomized(int(row['size']), int(row['no_comms']), row['inside_prob'], row['outside_prob'])
            # assor = nx.numeric_assortativity_coefficient(G, "community")
            # layout_names = ['kamada_kawai', 'spring', 'davidson_harel', 'drl', 'fruchterman_reingold', 'graphopt', 'lgl','mds']
            # for layout_name in layout_names:
            # posdf = posdf_from_layout(G, layout_name)
            layout_name='graphopt'
            posdf = self.graph_posdfs[row['graph_id']][layout_name]
            if best_num_algo_name == 'gap_statistic':
                (best_num, _) = gap_statistic_best_num(posdf)
            elif best_num_algo_name == 'elbow_method':
                (best_num, _) = elbow_method_best_num(posdf)
            elif best_num_algo_name == 'silhouette':
                (best_num, _) = silhouette_best_num(posdf)
            elif best_num_algo_name == 'calinski_harabasz':
                (best_num, _) = calinski_harabasz_best_num(posdf)
            elif best_num_algo_name == '50_mix_ch_elbow':
                best_num = mix_ch_elbow(posdf, .5, .5)
            elif best_num_algo_name == '75_mix_ch_elbow':
                best_num = mix_ch_elbow(posdf, .75, .25)
            elif best_num_algo_name == 'BIC':
                (best_num, _) = BIC_best_num(posdf)
            else:
                raise ValueError('Incorrect algorith name, probably a typo')

            new_row = {'graph_id':[int(row['graph_id'])], 
                        'assortativity': [self.graph_posdfs[row['graph_id']]['assortativity']], 
                        'size' : [int(row['size'])],
                        'layout_name': [layout_name], 
                        'no_communities': [int(row['no_comms'])],
                        'calculated_bestnum':[int(best_num)]}
            # print(new_row)
            results = pd.concat([results, pd.DataFrame(new_row)])
                
        if best_num_algo_name == 'gap_statistic':
            self.gap= results
        elif best_num_algo_name == 'elbow_method':
            self.elbow = results
        elif best_num_algo_name == 'silhouette':
            self.silhouette = results
        elif best_num_algo_name == 'calinski_harabasz':
            self.ch = results
        elif best_num_algo_name == '50_mix_ch_elbow':
            self.mix_ch_elbow_50 = results
        elif best_num_algo_name == '75_mix_ch_elbow':
            self.mix_ch_elbow_75 = results
        elif best_num_algo_name == 'BIC':
            self.bic = results


In [30]:
ex = NarrowedBestNumExperiment(graph_params)
ex.calculate_posdfs_grahopt()

ex.make_experiment('gap_statistic')
ex.gap.to_excel(fr'only_graphopt_experiment\graphopt_gap_results.xlsx', index=False)
ex.make_experiment('elbow_method')
ex.elbow.to_excel(fr'only_graphopt_experiment\graphopt_elbow_results.xlsx', index=False)
ex.make_experiment('silhouette')
ex.silhouette.to_excel(fr'only_graphopt_experiment\graphopt_silhouette_results.xlsx', index=False)
ex.make_experiment('calinski_harabasz')
ex.ch.to_excel(fr'only_graphopt_experiment\graphopt_ch_results.xlsx', index=False)
ex.make_experiment('50_mix_ch_elbow')
ex.mix_ch_elbow_50.to_excel(fr'only_graphopt_experiment\graphopt_50_mix_ch_elbow_results.xlsx', index=False)
ex.make_experiment('75_mix_ch_elbow')
ex.mix_ch_elbow_75.to_excel(fr'only_graphopt_experiment\graphopt_75_mix_ch_elbow_results.xlsx', index=False)

  2%|▏         | 1/66 [00:00<00:07,  8.96it/s]

0.0
1.0
2.0


  8%|▊         | 5/66 [00:00<00:04, 12.44it/s]

3.0
4.0
5.0
6.0


 14%|█▎        | 9/66 [00:00<00:04, 13.39it/s]

...

 97%|█████████▋| 64/66 [01:31<00:06,  3.35s/it]

64.0


 98%|█████████▊| 65/66 [01:34<00:03,  3.29s/it]

65.0


100%|██████████| 66/66 [00:38<00:00,  1.73it/s]


In [25]:
min(ex.ch['assortativity'])

0.5059779771210999

In [24]:
ex.ch

,graph_id,assortativity,layout_name,no_communities,calculated_bestnum,size
0,0,0.880479,graphopt,6,6,100.0
0,1,0.800390,graphopt,6,5,100.0
0,2,0.891334,graphopt,4,4,100.0
0,3,0.549456,graphopt,6,3,100.0
0,4,0.700072,graphopt,2,2,100.0
0,5,0.938109,graphopt,6,2,100.0
0,6,0.927512,graphopt,2,2,100.0
0,7,0.751499,graphopt,4,2,100.0
0,8,0.949236,graphopt,4,4,100.0
0,9,0.906524,graphopt,5,5,200.0
